# 1.1 Data

In [7]:
# Imports
import numpy as np
from glob import glob
from os import path
from scipy.special import softmax
from scipy.optimize import minimize, LinearConstraint, Bounds
from scipy.stats import binom

data_dir = "data"
file_paths = glob(path.join(data_dir, "*.txt"))

# N x M x K
# 5 FILES x 7 ROWS  x 5 COLUMNS
# ROW 1: Audio 
# ROW 2: Visual
# ROW 3-7: Visual going from 'b' (row 3) to 'd' (row 7)
# Columns: Audio from 'b' (col 1) to 'd' (col 5)
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

AUDIO_DATA       = data[:, 0, :]
VISUAL_DATA      = data[:, 1, :]
AUDIOVISUAL_DATA = data[:, 2:, :]

In [15]:
def objective_function(theta, data):
    theta_a = theta[0:K]
    theta_v = theta[K: ]

    p_a = softmax(theta_a).reshape(-1,1)
    p_v = softmax(theta_v).reshape(-1,1)
    
    # Outer product for all combinations
    p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)

    likelihoods = np.concatenate([
        binom.logpmf(data[0  ], 24, p_a ).flatten(), 
        binom.logpmf(data[1  ], 24, p_v ).flatten(), 
        binom.logpmf(data[2: ], 24, p_av).flatten()
    ]) 

    return -(likelihoods.sum())

theta = np.zeros(K*2)
opt_result = minimize(objective_function, theta, args=(data[0]))
objective, theta_a, theta_v, success = (
    opt_result.fun, 
    (opt_result.x[0:K]), 
    (opt_result.x[K:]), 
    opt_result.success
)

print("Converged:",success, "NLL:",objective)
print("theta_a", theta_a)
print("theta_v", theta_v)
p_a, p_v = softmax(theta_a).reshape(-1,1), softmax(theta_v).reshape(-1,1)
p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)
print("===")
print("p_a", p_a.flatten())
print("p_v", p_v.flatten())
print("p_av\n", p_av)
print("===")
print("data")
print(np.round(binom.pmf(data[0][2:], 24, p_av), 3))

Converged: False NLL: 1991.0090793646573
theta_a [-0.56077136 -0.12021155  0.16169979  0.2471653   0.27221288]
theta_v [-0.27719287 -0.12425869 -0.02973225  0.19507096  0.23623078]
===
p_a [0.10921155 0.16966852 0.22492298 0.2449915  0.25120544]
p_v [0.14879535 0.17338353 0.19057247 0.23861127 0.24863738]
p_av
 [[0.02098168 0.02507095 0.02805548 0.03700025 0.03898875]
 [0.0344876  0.04109864 0.04590133 0.06018349 0.06333598]
 [0.04827855 0.05737612 0.06395408 0.08336256 0.08761601]
 [0.05367767 0.06372453 0.07097551 0.09230469 0.09696613]
 [0.05539517 0.06574116 0.07320367 0.09513388 0.09992246]]
===
data
[[0.601 0.544 0.505 0.    0.   ]
 [0.007 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]]


In [9]:
theta = np.zeros(K*2)
opt_result = minimize(objective_function, theta, args=(data[1]))
objective, theta_a, theta_v, success = (
    opt_result.fun, 
    (opt_result.x[0:K]), 
    (opt_result.x[K:]), 
    opt_result.success
)

print("Converged:",success, "NLL:",objective)
print("theta_a", theta_a)
print("theta_v", theta_v)
p_a, p_v = softmax(theta_a).reshape(-1,1), softmax(theta_v).reshape(-1,1)
p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)
print("===")
print("p_a", p_a.flatten())
print("p_v", p_v.flatten())
print("p_av\n", p_av)
print("===")
print("data")
print(np.round(binom.pmf(data[1][2:], 24, p_av), 3))

Converged: True NLL: 28.84627507895132
theta_a [-0.72334057 -0.06968824  0.17459107  0.29796667  0.32048028]
theta_v [-0.23217787 -0.05956435  0.02787999  0.09794636  0.16592488]
===
p_a [0.09095919 0.17487358 0.22326125 0.25257752 0.25832845]
p_v [0.15709562 0.18669349 0.20375383 0.21854215 0.23391491]
p_av
 [[0.01830731 0.02245307 0.02496558 0.02722118 0.02964654]
 [0.0379984  0.04639255 0.05144292 0.05595344 0.06077885]
 [0.05084645 0.06189623 0.06851309 0.07440293 0.08068343]
 [0.05925002 0.07198753 0.07959165 0.08634559 0.09353233]
 [0.06095809 0.0740339  0.08183512 0.08876107 0.09612771]]
===
data
[[0.642 0.58  0.545 0.111 0.   ]
 [0.049 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]]


In [10]:
theta = np.zeros(K*2)
opt_result = minimize(objective_function, theta, args=(data[2]))
objective, theta_a, theta_v, success = (
    opt_result.fun, 
    (opt_result.x[0:K]), 
    (opt_result.x[K:]), 
    opt_result.success
)

print("Converged:",success, "NLL:",objective)
print("theta_a", theta_a)
print("theta_v", theta_v)
p_a, p_v = softmax(theta_a).reshape(-1,1), softmax(theta_v).reshape(-1,1)
p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)
print("===")
print("p_a", p_a.flatten())
print("p_v", p_v.flatten())
print("p_av\n", p_av)
print("===")
print("data")
print(np.round(binom.pmf(data[2][2:], 24, p_av), 3))

Converged: True NLL: 24.6852236107836
theta_a [-0.30287022 -0.1183694   0.06443915  0.15782099  0.19898032]
theta_v [-0.35483971 -0.23134403 -0.01870165  0.26421896  0.34066643]
===
p_a [0.1452769  0.17471261 0.20975725 0.23028848 0.23996476]
p_v [0.13526528 0.15304525 0.18930813 0.25121185 0.2711695 ]
p_av
 [[0.02589874 0.02979839 0.03817513 0.05394705 0.05947774]
 [0.03205338 0.0368447  0.04710609 0.0663134  0.07301406]
 [0.03986504 0.04576882 0.058365   0.08176915 0.0898812 ]
 [0.04470786 0.05129052 0.06530235 0.09121887 0.10016612]
 [0.04706318 0.05397304 0.06866466 0.09577884 0.10512162]]
===
data
[[0.34  0.484 0.05  0.    0.   ]
 [0.034 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.   ]]


In [11]:
theta = np.zeros(K*2)
opt_result = minimize(objective_function, theta, args=(data[3]))
objective, theta_a, theta_v, success = (
    opt_result.fun, 
    (opt_result.x[0:K]), 
    (opt_result.x[K:]), 
    opt_result.success
)

print("Converged:",success, "NLL:",objective)
print("theta_a", theta_a)
print("theta_v", theta_v)
p_a, p_v = softmax(theta_a).reshape(-1,1), softmax(theta_v).reshape(-1,1)
p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)
print("===")
print("p_a", p_a.flatten())
print("p_v", p_v.flatten())
print("p_av\n", p_av)
print("===")
print("data")
print(np.round(binom.pmf(data[3][2:], 24, p_av), 3))

Converged: True NLL: 16.93029051212122
theta_a [-0.20306949 -0.09684237 -0.02189728  0.08425744  0.23755078]
theta_v [-0.54336898 -0.42635208 -0.24803157  0.51632618  0.70142478]
===
p_a [0.16136251 0.17944712 0.19341258 0.21507361 0.25070418]
p_v [0.10177674 0.11441114 0.13674515 0.29367589 0.35339108]
p_av
 [[0.02133663 0.02425498 0.02957754 0.07407449 0.09515199]
 [0.02418041 0.02747679 0.0334821  0.08334861 0.10676075]
 [0.02645178 0.03004828 0.03659446 0.09066154 0.11586818]
 [0.0301123  0.03418901 0.04159858 0.10227421 0.13024708]
 [0.03652684 0.04143488 0.05033297 0.12212516 0.15459245]]
===
data
[[0.596 0.331 0.125 0.    0.   ]
 [0.556 0.512 0.442 0.    0.   ]
 [0.526 0.481 0.163 0.    0.   ]
 [0.358 0.007 0.    0.    0.   ]
 [0.009 0.    0.    0.    0.   ]]


In [12]:
theta = np.zeros(K*2)
opt_result = minimize(objective_function, theta, args=(data[4]))
objective, theta_a, theta_v, success = (
    opt_result.fun, 
    (opt_result.x[0:K]), 
    (opt_result.x[K:]), 
    opt_result.success
)

print("Converged:",success, "NLL:",objective)
print("theta_a", theta_a)
print("theta_v", theta_v)
p_a, p_v = softmax(theta_a).reshape(-1,1), softmax(theta_v).reshape(-1,1)
p_av = (p_a @ p_v.T) / (p_a @ p_v.T + (1 - p_a) @ (1 - p_v).T)
print("===")
print("p_a", p_a.flatten())
print("p_v", p_v.flatten())
print("p_av\n", p_av)
print("===")
print("data")
print(np.round(binom.pmf(data[4][2:], 24, p_av), 3))

Converged: True NLL: 23.521326968804473
theta_a [-0.25710474 -0.05043896  0.02310099  0.11330089  0.17114261]
theta_v [-0.50995773 -0.39234804  0.00719336  0.39722598  0.4978856 ]
===
p_a [0.15300203 0.18812694 0.20248319 0.2215962  0.23479165]
p_v [0.11087618 0.12471409 0.18596625 0.27467754 0.30376594]
p_av
 [[0.02203009 0.02509249 0.03963184 0.06402797 0.07305541]
 [0.02808454 0.03196107 0.05027504 0.08067243 0.09181641]
 [0.03068938 0.03491254 0.05482195 0.0877146  0.09972584]
 [0.03428334 0.03898117 0.06106398 0.09731618 0.11048299]
 [0.03685289 0.04188755 0.0655046  0.10410081 0.11806551]]
===
data
[[0.586 0.543 0.375 0.    0.   ]
 [0.35  0.363 0.005 0.    0.   ]
 [0.473 0.154 0.    0.    0.   ]
 [0.151 0.011 0.    0.    0.   ]
 [0.373 0.    0.    0.    0.   ]]
